In [1]:
from torchcnnbuilder.models import ForecasterBase
from torchcnnbuilder.preprocess.time_series import multi_output_tensor

import os
from datetime import datetime

from tqdm.notebook import tqdm

import numpy as np

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [2]:
device = 'mps'
print(f'Calculating on device: {device}')

Calculating on device: mps


In [3]:
batch_size = 20
epochs = 300
learning_rate = 1e-2

data_freq = 7

In [4]:
x_virg = []
temp_ar = []
for file in os.listdir('/Users/kdduha/projects/osisaf/train'):
    date = datetime.strptime(file, 'osi_iceconc_%Y%m%d.npy')
    if date.year < 2013:
        array = np.load(f'/Users/kdduha/projects/osisaf/train/{file}')
        temp_ar.append(array)
        if len(temp_ar) == data_freq:
            temp_ar = np.array(temp_ar)
            temp_ar = temp_ar[-1]
            x_virg.append(temp_ar)
            temp_ar = []

In [5]:
for file in os.listdir('/Users/kdduha/projects/osisaf/test'):
    date = datetime.strptime(file, 'osi_iceconc_%Y%m%d.npy')
    if date.year < 2013:
        array = np.load(f'../../matrices/osisaf/test/{file}')
        temp_ar.append(array)
        if len(temp_ar) == data_freq:
            temp_ar = np.array(temp_ar)
            temp_ar = temp_ar[-1]
            x_virg.append(temp_ar)
            temp_ar = []

In [6]:
x_virg = np.array(x_virg)
x_virg.shape

(1200, 125, 125)

In [7]:
pre_history_size = 104
forecast_size = 52

In [8]:
dataset = multi_output_tensor(data=x_virg,
                             forecast_len=forecast_size,
                             pre_history_len=pre_history_size)

In [9]:
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [10]:
model = ForecasterBase(input_size=[125, 125],
                       n_layers=5,
                       in_channels=pre_history_size,
                       out_channels=forecast_size).to(device)

In [11]:
model

ForecasterBase(
  (convolve): Sequential(
    (conv 1): Sequential(
      (0): Conv2d(104, 116, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU(inplace=True)
    )
    (conv 2): Sequential(
      (0): Conv2d(116, 128, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU(inplace=True)
    )
    (conv 3): Sequential(
      (0): Conv2d(128, 140, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU(inplace=True)
    )
    (conv 4): Sequential(
      (0): Conv2d(140, 152, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU(inplace=True)
    )
    (conv 5): Sequential(
      (0): Conv2d(152, 164, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU(inplace=True)
    )
  )
  (transpose): Sequential(
    (deconv 1): Sequential(
      (0): ConvTranspose2d(164, 162, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU(inplace=True)
    )
    (deconv 2): Sequential(
      (0): ConvTranspose2d(162, 140, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU(inplace=True)
    )
    (deconv 3): Sequential(
  

In [12]:
# добавить возможность смотреть слои и т.д.

In [ ]:
model.builder

In [13]:
from torchcnnbuilder.builder import EncoderBuilder

In [15]:
builder = EncoderBuilder(input_size=[125, 125])

In [16]:
d = builder.build_convolve_sequence(n_layers=5,
                                                        in_channels=pre_history_size,
                                                        ascending=True)

In [17]:
d

Sequential(
  (conv 1): Sequential(
    (0): Conv2d(104, 116, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (conv 2): Sequential(
    (0): Conv2d(116, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (conv 3): Sequential(
    (0): Conv2d(128, 140, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (conv 4): Sequential(
    (0): Conv2d(140, 152, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (conv 5): Sequential(
    (0): Conv2d(152, 164, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
)

In [18]:
builder.conv_layer_sizes

[[125, 125], (123, 123), (121, 121), (119, 119), (117, 117), (115, 115)]

In [20]:
d = builder.build_transpose_convolve_sequence(n_layers=5,
                                            in_channels=builder.conv_channels[-1],
                                            out_channels=52,
                                            ascending=True)

In [21]:
d

Sequential(
  (deconv 1): Sequential(
    (0): ConvTranspose2d(164, 162, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (deconv 2): Sequential(
    (0): ConvTranspose2d(162, 140, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (deconv 3): Sequential(
    (0): ConvTranspose2d(140, 118, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (deconv 4): Sequential(
    (0): ConvTranspose2d(118, 96, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (deconv 5): Sequential(
    (0): ConvTranspose2d(96, 52, kernel_size=(3, 3), stride=(1, 1))
  )
  (resize): AdaptiveAvgPool2d(output_size=(125, 125))
)

In [22]:
builder.transpose_conv_layer_sizes

[(115, 115), (117, 117), (119, 119), (121, 121), (123, 123), (125, 125)]

In [15]:

#optimizer = optim.Adam(model.parameters(), lr=learning_rate)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience = 5)
criterion = nn.L1Loss()

losses = []
epoches = []

for epoch in tqdm(range(epochs)):
    loss = 0
    for train_features, test_features in dataloader:
        train_features = train_features.to(device)
        test_features = test_features.to(device)
        
        optimizer.zero_grad()
        outputs = model(train_features)
    
        l1_loss = criterion(outputs, test_features)
        train_loss = l1_loss
        
        train_loss.backward()
        optimizer.step()
        
        loss += train_loss.item()
    
    loss = loss / len(dataloader)
    scheduler.step(loss)
    
    losses.append(loss)
    epoches.append(epoch)
    
    tqdm.write("epoch : {}/{}, recon loss = {:.8f}".format(epoch + 1, epochs, loss))

  0%|          | 0/300 [00:00<?, ?it/s]

epoch : 1/300, recon loss = 0.21468056
epoch : 2/300, recon loss = 0.21464311
epoch : 3/300, recon loss = 0.21463899
epoch : 4/300, recon loss = 0.21463627
epoch : 5/300, recon loss = 0.21463528
epoch : 6/300, recon loss = 0.21463485
epoch : 7/300, recon loss = 0.21463320


KeyboardInterrupt: 

In [ ]:
plt.plot(epoches, losses, label='Loss history')
plt.grid()
plt.xlabel('Epoch')
plt.ylabel('L1Loss')
plt.legend()
plt.show()

Вроде forward и backward работают адекватно, проверял модель с нормализацией (последний запуск) + без нормализации как в статье (первичный запуск), довел до эпох 40 и бросил 